In [1]:
%matplotlib notebook

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import requests

pd.set_option('display.max_rows', 500)

In [3]:
covid_df = pd.read_csv(os.path.join("COVID","03-16-2020.csv"))
world_pop_df = pd.read_csv(os.path.join('GDP', 'world_countries_gdp.csv'))
median_age_df = pd.read_csv(os.path.join('Median Age', 'output','median_age.csv'))
life_expectancy = pd.read_csv(os.path.join('Life Expectancy','output','life_expectancy.csv'))
unemployment_rate = pd.read_csv(os.path.join('Unemployment Rate','output','unemployment_rate.csv'))
medical_expenditure = pd.read_csv(os.path.join("Medical Expenditure", "NHA indicators.csv"),encoding = 'ISO-8859-1')
poverty = pd.read_csv(os.path.join('Poverty','output','poverty.csv'))
literacy = pd.read_csv(os.path.join('Literacy_Rate','literacy_rate.csv'))
bed_density = pd.read_csv(os.path.join('Hospital_beds','beds_density.csv'))
gender_ratio = pd.read_csv(os.path.join('Gender Ratio','output','gender_ratio.csv'))

#number of countries in each csv

print(f"COVID-19 Countries: {covid_df['Country/Region'].nunique()}")
print(f"World Population Countries: {world_pop_df['country'].nunique()}")
print(f"Median Age Countries: {median_age_df['Country'].nunique()}")
print(f"Life Expectancy Countries: {life_expectancy['Country'].nunique()}")
print(f"Unemployment Rate Countries: {unemployment_rate['Country'].nunique()}")
print(f"Medical Expenditure Countries: {medical_expenditure['Countries'].nunique()}")
print(f"Poverty Countries: {poverty['Country'].nunique()}")
print(f"Literacy Countries: {literacy['Country'].nunique()}")
print(f"Bed Density Countries: {bed_density['Country'].nunique()}")
print(f"Gender Ratio Countries: {gender_ratio['Country'].nunique()}")


COVID-19 Countries: 156
World Population Countries: 211
Median Age Countries: 259
Life Expectancy Countries: 259
Unemployment Rate Countries: 259
Medical Expenditure Countries: 188
Poverty Countries: 259
Literacy Countries: 259
Bed Density Countries: 259
Gender Ratio Countries: 259


In [4]:
#get date only
#map takes in a function. So we use a temporary function (lambda)
covid_df['Last Update'] = covid_df['Last Update'].map(lambda x: x[0:10])

In [5]:
#clean data
grouped_covid_df = covid_df.groupby(['Country/Region']).sum()
grouped_covid_df.drop(['Latitude','Longitude'],1,inplace=True)
grouped_covid_df.rename(index = {"US":"United States","Taiwan*":"Taiwan"}, inplace=True)
grouped_covid_df.reset_index(inplace=True)
grouped_covid_df.rename(columns = {"Country/Region":"Country"}, inplace=True)
medical_expenditure.columns = ['Countries', '2017 Health Budget per Capita ($)']
#grouped_covid_df.query('Country_Region.str.contains("Congo")',engine='python',inplace=True)

#clean medical expenditure
medical_expenditure.rename(columns = {'Countries': 'Country'},inplace=True)

In [6]:
#clean data
grouped_world_pop_df = world_pop_df.groupby('country').sum()
grouped_world_pop_df.drop(['rank','unGDP'],1,inplace=True)
grouped_world_pop_df.rename(index = {"South Korea":"Korea, South","Bahamas":"The Bahamas"}, 
                            columns = {"imfGDP": "GDP","gdpPerCapita": "GDP Per Capita","pop":"Population"},
                            inplace=True)
grouped_world_pop_df.index.name = 'Country'
grouped_world_pop_df.reset_index(inplace=True)

In [7]:
clean_median_age_df = median_age_df.copy()
#replace country column with string replace
#clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='And',repl='and',case=False)
#clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='The',repl='the',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='Congo, Democratic Republic Of the',repl='Republic of the Congo',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='andorra',repl='Andorra',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='Bahamas, the',repl='The Bahamas',case=False)
clean_median_age_df.sort_values(by='Country',ascending=True,inplace=True)
clean_median_age_df.reset_index(drop=True,inplace=True)

In [8]:
merged_df = grouped_covid_df.merge(grouped_world_pop_df,how='outer',on='Country')
#merged_df.dropna(inplace=True)
merged_df.reset_index(drop=True,inplace=True)

In [9]:
complete_df = merged_df.merge(clean_median_age_df,how='outer',on='Country')
complete_df.sort_values(by='Country',ascending=True,inplace=True)
complete_df.reset_index(drop=True,inplace=True)
#complete_df.dropna(inplace=True)
#complete_df.reset_index(drop=True,inplace=True)

complete_df.head()

,Country,Confirmed,Deaths,Recovered,GDP,GDP Per Capita,Population,Median Age
0,Afghanistan,21.0,0.0,1.0,2.068200e+10,531.2838,38928.346,19.0
1,Akrotiri,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,51.0,1.0,0.0,1.721000e+10,5980.2689,2877.797,33.4
3,Algeria,54.0,4.0,12.0,1.930560e+11,4402.5406,43851.044,28.3
4,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,26.1


In [10]:
complete_df.to_csv('Final Data\complete_data.csv', index=False)

In [11]:
target_url = "https://raw.githubusercontent.com/iancoleman/cia_world_factbook_api/master/data/factbook.json"
cia = requests.get(target_url).json()

In [12]:
beds = {}

for country in cia['countries'].keys():
    try:
        beds[cia['countries'][country]['data']['name']] = cia['countries'][country]['data']['people']['hospital_bed_density']['beds_per_1000_population']
    except Exception as ex:
        #print(f"{cia['countries'][country]['data']['name']} does not have data")
        #print(ex)   
        beds[cia['countries'][country]['data']['name']] = ''

Beds_df = pd.DataFrame(beds, index=[0])
Beds_df = Beds_df.transpose()
Beds_df.reset_index(inplace=True)
Beds_df.rename(columns={'index':'Country', 0:'Hospital Bed Per 1000 Population Rate'},inplace=True)
Beds_df.head()

,Country,Hospital Bed Per 1000 Population Rate
0,World,
1,Afghanistan,0.5
2,Akrotiri,
3,Albania,2.9
4,Algeria,1.9


In [13]:
literacy = {}

for country in cia['countries'].keys():
    try:
        literacy[cia['countries'][country]['data']['name']] = cia['countries'][country]['data']['people']['literacy']['total_population']['value']
    except Exception as ex:
        literacy[cia['countries'][country]['data']['name']] = ''

literacy_df = pd.DataFrame(literacy,index=[0])
literacy_df = literacy_df.transpose()
literacy_df.reset_index(inplace=True)
literacy_df.rename(columns={'index':'Country', 0:'Literacy Rate'},inplace=True)
literacy_df.head()

,Country,Literacy Rate
0,World,86.2
1,Afghanistan,38.2
2,Akrotiri,
3,Albania,97.6
4,Algeria,80.2


In [14]:
Beds_df.to_csv('beds_density.csv', index=False)

In [15]:
literacy_df.to_csv('literacy_rate.csv', index=False)

In [16]:
#merge complete_df with bed_density
df_list = [Beds_df,life_expectancy,unemployment_rate,medical_expenditure,poverty,literacy_df,gender_ratio]

for df in df_list:
    complete_df = complete_df.merge(df,how='outer',on='Country')

In [17]:
complete_df.set_index('Country',inplace=True)

In [18]:
complete_df.rename(index={'Congo (Brazzaville)': 'Congo, Republic Of The',
                          'DR Congo': 'Congo, Democratic Republic Of The',
                          'Congo': 'Congo, Republic Of The',
                          'Congo (Kinshasa)': 'Congo, Democratic Republic Of The',
                          'Democratic Republic of the Congo': 'Congo, Democratic Republic Of The',
                          'Republic of the Congo': 'Congo, Democratic Republic Of The',
                          'Congo (Kinshasa)': 'Congo, Democratic Republic Of The'},inplace=True)

complete_df.reset_index(inplace=True)

In [27]:
#fix this garbage data
complete_df['Country'] = complete_df['Country'].str.replace(pat=' and ',repl=' And ',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='of ',repl='Of ',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='United States of America',repl='United States',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='the ',repl='The ',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='the ',repl='The ',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat=', The',repl='',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='The Bahamas',repl='Bahamas',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Bolivia Plurinational States Of',repl='Bolivia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Brunei Darussalam',repl='Brunei',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cabo Verde',repl='Cape Verde',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cape Verde Republic of',repl='Cape Verde',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cote D\'Ivoire',repl='Ivory Coast',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cte d\'Ivoire',repl='Ivory Coast',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cote d\'Ivoire',repl='Ivory Coast',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cote d\'Ivoire',repl='Ivory Coast',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Cote d\'Ivoire',repl='Ivory Coast',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Lao People\'s Democratic Republic',repl='Laos',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Micronesia \(Federated States of\)',repl='Micronesia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Micronesia, Federated States Of',repl='Micronesia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Burma',repl='Myanmar',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='North Korea',repl='Korea, North',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Russian Federation',repl='Russia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='North Macedonia',repl='Macedonia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='The Republic Of Macedonia',repl='Macedonia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Venezuela \(Bolivarian Republic of\)',repl='Venezuela',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Viet Nam',repl='Vietnam',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='United Republic Of Tanzania',repl='Tanzania',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Bolivia ',repl='Bolivia',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Republic Of Moldova',repl='Moldova',case=False)
complete_df['Country'] = complete_df['Country'].str.replace(pat='Republic Of Korea',repl='Korea, South',case=False)

In [20]:
#drop occupied Palestinian territory, holy see, oceans, and cruise
complete_df.drop([287,114,115,11,16,35,121,193,240,66,283],axis=0,inplace=True)

In [39]:
complete_df.query('Country.str.contains("Bolivia")',engine='python')

,Country,Confirmed,Deaths,Recovered,GDP,GDP Per Capita,Population,Median Age,Hospital Bed Per 1000 Population Rate,Life Expectancy (years),Unemployment Rate (%),2017 Health Budget per Capita ($),Poverty (% of Pop),Literacy Rate,Gender (% of male)
29,Bolivia,11.0,0.0,0.0,4.705200e+10,4030.8331,11673.021,24.6,1.1,69.8,4.0,NaN,38.6,92.5,0.4949
297,Bolivia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220,NaN,NaN,NaN


In [33]:
aggregation_functions = {'Confirmed': 'sum',
                         'Deaths': 'sum',
                         'Recovered': 'sum',
                         'GDP': 'sum',
                         'GDP Per Capita': 'sum',
                         'Population': 'sum',
                         'Median Age': 'sum',
                         'Hospital Bed Per 1000 Population Rate': 'sum',
                         'Life Expectancy (years)': 'sum',
                         'Unemployment Rate (%)': 'sum',
                         '2017 Health Budget per Capita ($)': 'sum',
                         'Poverty (% of Pop)': 'sum',
                         'Literacy Rate': 'sum',
                         'Gender (% of male)': 'sum'}

d = complete_df.copy()
d.fillna(0,inplace=True)
#replace blanks and change datatype to float
d['Hospital Bed Per 1000 Population Rate'] = d['Hospital Bed Per 1000 Population Rate'].replace('',0)
d['Literacy Rate'] = d['Literacy Rate'].replace('',0)
d['2017 Health Budget per Capita ($)'] = d['2017 Health Budget per Capita ($)'].replace('-',0)
d['2017 Health Budget per Capita ($)'] = d['2017 Health Budget per Capita ($)'].astype('float64')

In [34]:
#d.set_index('Country',inplace=True)
d = d.groupby('Country').aggregate(aggregation_functions)

In [38]:
#d['GDP Per Capita'] = d['GDP Per Capita'].map('{:.0f}'.format)
d['Population'] = (d['Population'] * 1000).astype('int64')
d.sort_values(by='Confirmed',ascending=False)

,Confirmed,Deaths,Recovered,GDP,GDP Per Capita,Population,Median Age,Hospital Bed Per 1000 Population Rate,Life Expectancy (years),Unemployment Rate (%),2017 Health Budget per Capita ($),Poverty (% of Pop),Literacy Rate,Gender (% of male)
Country,,,,,,,,,,,,,,
China,81033.0,3217.0,67910.0,1.546810e+13,10746.7828,1439323776000000,37.7,4.2,75.80,3.9,441.0,3.3,96.4,0.5146
Italy,27980.0,2158.0,2749.0,2.090450e+12,34574.7083,60461826000000,45.8,3.4,82.40,11.3,2840.0,29.9,99.2,0.4819
Iran,14991.0,853.0,4590.0,4.956940e+11,5901.6144,83992949000000,30.8,0.2,74.20,11.8,475.0,18.7,85.5,0.5074
Spain,9942.0,342.0,530.0,1.497080e+12,32019.8291,46754778000000,43.1,3.0,81.80,17.2,2506.0,21.1,98.3,0.4949
"Korea, South",8236.0,75.0,1137.0,1.744360e+12,34023.5563,51269185000000,42.3,11.5,82.50,3.7,2283.0,14.4,0.0,0.5000
Germany,7272.0,17.0,67.0,4.157120e+12,49617.1450,83783942000000,47.4,8.3,80.90,3.8,5033.0,16.7,0.0,0.4898
France,6650.0,148.0,12.0,2.876050e+12,44061.5183,65273511000000,41.5,6.5,82.00,9.4,4380.0,14.2,0.0,0.4898
United States,4632.0,85.0,17.0,2.219812e+13,67063.2695,331002651000000,38.2,2.9,80.10,4.4,10246.0,15.1,0.0,0.4924
Switzerland,2200.0,14.0,4.0,7.407020e+11,85584.5582,8654622000000,42.5,4.7,82.70,3.2,9956.0,6.6,0.0,0.4924


In [25]:
d.query('Country.str.contains("Congo")',engine='python')

,Confirmed,Deaths,Recovered,GDP,GDP Per Capita,Population,Median Age,Hospital Bed Per 1000 Population Rate,Life Expectancy (years),Unemployment Rate (%),2017 Health Budget per Capita ($),Poverty (% of Pop),Literacy Rate,Gender (% of male)
Country,,,,,,,,,,,,,,
"Congo, Democratic Republic Of The",3.0,0.0,0.0,6.384200e+10,2645.8639,95079.49,18.8,0.0,58.1,0.0,19.0,63.0,77.0,0.5000
"Congo, Republic Of The",1.0,0.0,0.0,0.000000e+00,0.0000,0.00,19.6,0.0,60.3,36.0,50.0,46.5,79.3,0.5025
